In [24]:
import pickle
import os 

pickle_file = open("data/car_number_1.pkl", "rb")
unpickled = pickle.load(pickle_file)

basic_info = unpickled[0]
basic_info.update(unpickled[1])
additional_info = unpickled[2]

print("=== BASIC INFO ===\n", basic_info.keys())
print("\n=== ADDITIONAL INFO ===\n", additional_info)

=== BASIC INFO ===
 dict_keys(['url', 'image', 'brand', 'model', 'manufacturer', 'productionDate', 'fuelType', 'bodyType', 'vehicleEngine', 'mileageFromOdometer', 'name', 'Plivajući zamajac', 'Emisiona klasa motora', 'Pogon', 'Menjač', 'Broj vrata', 'Broj sedišta', 'Strana volana', 'Klima', 'Boja', 'Materijal enterijera', 'Boja enterijera', 'Registrovan do', 'Poreklo vozila', 'Oštećenje'])

=== ADDITIONAL INFO ===
 ['Airbag za vozača', 'Airbag za suvozača', 'Bočni airbag', 'ABS', 'ESP', 'Kodiran ključ', 'Blokada motora', 'Centralno zaključavanje', 'Metalik boja', 'Branici u boji auta', 'Servo volan', 'Multifunkcionalni volan', 'Tempomat', 'Daljinsko zaključavanje', 'Putni računar', 'Tonirana stakla', 'Električni podizači', 'Električni retrovizori', 'Sedišta podesiva po visini', 'Svetla za maglu', 'Senzori za svetla', 'Senzori za kišu', 'Parking senzori', 'Aluminijumske felne', 'Navigacija', 'Radio CD', 'Naslon za ruku', 'Hands free', 'Kožni volan', 'MP3', 'Utičnica od 12V', 'Otvor za s

In [41]:
def generate_price(basic_info, scale=1e6):
    
    mileage_str_raw = basic_info['mileageFromOdometer'] 
    mileage_str = mileage_str_raw[ : -5]
    mileage = float(mileage_str)

    return mileage


raw_data = []

data_folder = "data"
file_names = os.listdir(data_folder)

for file_name in file_names:

    file_path = os.path.join(data_folder, file_name)git
    pickle_file = open(file_path, "rb")
    unpickled = pickle.load(pickle_file)

    basic_info = unpickled[0]
    basic_info.update(unpickled[1])
    additional_info = unpickled[2]

    price = generate_price(basic_info)

    raw_data.append({
        'features': basic_info,
        'price': price 
        })

    break

raw_data